<a href="https://colab.research.google.com/github/sadullahmath/Exercise/blob/master/Adding_Data_Processing_Logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adding Data Processing Logic


In [0]:
import pandas as pd
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter03/bank-full.csv'
df = pd.read_csv(file_url, sep=';')

In [0]:
y = df.pop('y')

In [0]:
X_train = df[:31648]
y_train = y[:31648]
X_test = df[31648:]
y_test = y[31648:]

In [5]:
print(sorted(X_train['month'].unique()))
print(sorted(X_test['month'].unique()))

['apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov', 'oct']
['apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep']


In [6]:
cat_columns = [col for col in X_train.columns if X_train[col].dtype == 'object']
cat_columns

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome']

In [0]:
column_categories = {}
for col in cat_columns:
  column_categories[col] = X_train[col].astype('category').cat.categories

In [8]:
column_categories

{'contact': Index(['cellular', 'telephone', 'unknown'], dtype='object'),
 'default': Index(['no', 'yes'], dtype='object'),
 'education': Index(['primary', 'secondary', 'tertiary', 'unknown'], dtype='object'),
 'housing': Index(['no', 'yes'], dtype='object'),
 'job': Index(['admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management',
        'retired', 'self-employed', 'services', 'student', 'technician',
        'unemployed', 'unknown'],
       dtype='object'),
 'loan': Index(['no', 'yes'], dtype='object'),
 'marital': Index(['divorced', 'married', 'single'], dtype='object'),
 'month': Index(['apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov',
        'oct'],
       dtype='object'),
 'poutcome': Index(['failure', 'other', 'success', 'unknown'], dtype='object')}

In [9]:
from pandas.api.types import CategoricalDtype
for col, cat in column_categories.items():
  X_train[col] = X_train[col].astype(CategoricalDtype(categories=cat))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
X_train_final = pd.get_dummies(X_train, columns=cat_columns)

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=8)
rf_model.fit(X_train_final, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=8, verbose=0,
                       warm_start=False)

In [0]:
import pickle
pickle.dump(column_categories, open("categories_data.pkl", "wb" ) )

In [13]:
categories_data = pickle.load(open("categories_data.pkl", "rb" ) )
categories_data

{'contact': Index(['cellular', 'telephone', 'unknown'], dtype='object'),
 'default': Index(['no', 'yes'], dtype='object'),
 'education': Index(['primary', 'secondary', 'tertiary', 'unknown'], dtype='object'),
 'housing': Index(['no', 'yes'], dtype='object'),
 'job': Index(['admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management',
        'retired', 'self-employed', 'services', 'student', 'technician',
        'unemployed', 'unknown'],
       dtype='object'),
 'loan': Index(['no', 'yes'], dtype='object'),
 'marital': Index(['divorced', 'married', 'single'], dtype='object'),
 'month': Index(['apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov',
        'oct'],
       dtype='object'),
 'poutcome': Index(['failure', 'other', 'success', 'unknown'], dtype='object')}

In [14]:
record = X_test[X_test['month'] == 'sep'].iloc[0].to_json()
record

'{"age":23,"job":"student","marital":"single","education":"secondary","default":"no","balance":922,"housing":"no","loan":"no","contact":"cellular","day":1,"month":"sep","duration":81,"campaign":5,"pdays":99,"previous":3,"poutcome":"other"}'

In [0]:
import json
j_data = json.loads(record)

In [16]:
df_test = pd.DataFrame(j_data, index=[0])
df_test

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,23,student,single,secondary,no,922,no,no,cellular,1,sep,81,5,99,3,other


In [0]:
for col, cat in categories_data.items():
  df_test[col] = df_test[col].astype(CategoricalDtype(categories=cat))

In [0]:
df_test_final = pd.get_dummies(df_test, columns=cat_columns)

In [23]:
rf_model.predict(df_test_final)

array(['no'], dtype=object)